---
title: Saving and Loading
jupyter: python3
eval: true
---


This tutorial shows how to save and load objects in `spotpython`.
It is split into the following parts:

* @sec-spotpython-saving-and-loading shows how to save and load objects in `spotpython`, if `spotpython` is used as an optimizer.
* @sec-spotpython-as-a-hyperparameter-tuner-604 shows how to save and load hyperparameter tuning experiments.
* @sec-saving-and-loading-pytorch-lightning-models-604 shows how to save and load `PyTorch Lightning` models.
* @sec-converting-a-lightning-model-to-a-plain-torch-model-604 shows how to convert a `PyTorch Lightning` model to a plain `PyTorch` model.

## Required Python Packages

In [ ]:
#| label: code-required-packages
import os
import pprint
import numpy as np
from math import inf
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torch.utils.data import random_split
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from spotpython.utils.file import (load_experiment, load_result, get_experiment_filename, load_and_run_spot_python_experiment)
from spotpython.spot import Spot
from spotpython.utils.init import (
    fun_control_init,
    design_control_init,
    surrogate_control_init,
    optimizer_control_init)
from spotpython.fun.objectivefunctions import Analytical
from spotpython.hyperparameters.values import get_tuned_hyperparameters
from spotpython.data.diabetes import Diabetes
from spotpython.hyperdict.light_hyper_dict import LightHyperDict
from spotpython.fun.hyperlight import HyperLight
from spotpython.utils.eda import print_exp_table
from spotpython.hyperparameters.values import set_hyperparameter, get_tuned_architecture
from spotpython.light.testmodel import test_model
from spotpython.light.loadmodel import load_light_from_checkpoint
from spotpython.utils.device import getDevice
from spotpython.utils.classes import get_removed_attributes_and_base_net
from spotpython.hyperparameters.optimizer import optimizer_handler

And we will suppress warnings in this notebook.

In [ ]:
#| label: warnings-off-604_12
#| echo: false
import warnings
warnings.filterwarnings("ignore")

## Quick Overview of `spotpython` Save and Load Functions

Because the most common use case is to save and load hyperparameter tuning experiments, the `save_experiment` and `load_experiment` functions are explained first. In the real- world setting, users specify an experiment on their local machine and then run the experiment on a remote machine. Then, the result file is copied back to the local machine and loaded for further analysis.
So, the main steps can be summarized as follows:

1. Generate an experiment configuration on the local machine, say `42_exp.pkl`. To generate the configuration, the `fun_control` dictionary has to be initialized with `save_experiment=True`. 
2. Copy the configuration file to the remote machine.
3. Run the experiment on the remote machine. The run on the remote machine is started with the command `load_and_run_spot_python_experiment(filename="42_exp.pkl")`. This generates a result file, say `42_res.pkl`.
4. Copy the result file back to the local machine. Analyze the results on the local machine as shown below.

As can be seen from these step, we will distinguish between experiment/design/configuration files (with `_exp` in the filename) and result files (with `_res` in the filename).

::: {#exm-spotpython-save-load}
## Save and Load Hyperparameter Tuning Experiment

1. Generate an experiment configuration on the local machine, say `42_exp.pkl` as follows:

In [ ]:
#| label: code-save-experiment-604_1
PREFIX = "42"
data_set = Diabetes()
fun_control = fun_control_init(
    save_experiment=True,
    PREFIX=PREFIX,
    fun_evals=inf,
    max_time=1,
    data_set = data_set,
    core_model_name="light.regression.NNLinearRegressor",
    hyperdict=LightHyperDict,
    _L_in=10,
    _L_out=1)

fun = HyperLight().fun

set_hyperparameter(fun_control, "optimizer", [ "Adadelta", "Adam", "Adamax"])
set_hyperparameter(fun_control, "l1", [3,4])
set_hyperparameter(fun_control, "epochs", [3,5])
set_hyperparameter(fun_control, "batch_size", [4,11])
set_hyperparameter(fun_control, "dropout_prob", [0.0, 0.025])
set_hyperparameter(fun_control, "patience", [2,3])

design_control = design_control_init(init_size=10)

print_exp_table(fun_control)

S = Spot(fun=fun,fun_control=fun_control, design_control=design_control)

As the output shows, the configuration is saved as a pickle-file that contains the full information. In our example, the filename is `42_exp.pkl`.

2. Copy the configuration file to the remote machine. This can be done with the `scp` command, see below, but is omitted here for brevity.

3. Run the experiment on the remote machine. This step is simulated on the local machine for demonstration purposes. This generates a result file, say `42_res.pkl`. This can be done with the following code:

In [ ]:
#| label: code-run-experiment-604_2
# S_res = S.run() is NOT used here, because we want to simulate the remote run
# Instead, we use the load_and_run_spot_python_experiment function:
S_remote = load_and_run_spot_python_experiment(filename="42_exp.pkl")

A result file with the name `42_res.pkl` is stored in the current directory.

4. Copy the result file back to the local machine. This can be done with the `scp` command, see below, but is omitted here for brevity. The result file can be loaded with the following code:

In [ ]:
#| label: code-load-result-604_1
S_res = load_result(PREFIX="42")

In [ ]:
#| label: code-get-tuned-hyperparameters-fun-ctrl604
S_res.plot_progress(log_y=True)

In [ ]:
S_res.print_results()

If you add `fun_control=S_res.fun_control` as an argument to the `get_tuned_hyperparameters` function, the names of the hyperparameters are used as keys in the dictionary.

In [ ]:
get_tuned_hyperparameters(S_res, fun_control=S_res.fun_control)

Get the transformed hyperparameters that can be passed to the model.

In [ ]:
config = get_tuned_architecture(S_res)
pprint.pprint(config)

After getting the tuned architecture, the model can be created and tested with the following code.

In [ ]:
test_model(config, S_res.fun_control)

In [ ]:
S_res.plot_important_hyperparameter_contour(max_imp=3)

In [ ]:
S_res.plot_importance()

:::





## spotpython: Saving and Loading Optimization Experiments {#sec-spotpython-saving-and-loading}

In this section, we will show how results from `spotpython` can be saved and reloaded.
Here, `spotpython` can be used as an optimizer. 
If `spotpython` is used as an optimizer, no dictionary of hyperparameters has be specified.
The `fun_control` dictionary is sufficient. 


In [ ]:
#| label: code-optimization-experiment-604_1
PREFIX="branin"

fun = Analytical().fun_branin
fun_control = fun_control_init(
            PREFIX=PREFIX,
            lower = np.array([0, 0]),
            upper = np.array([10, 10]),
            fun_evals=8,
            fun_repeats=1,
            max_time=inf,
            noise=False,
            tolerance_x=0,
            ocba_delta=0,
            var_type=["num", "num"],
            infill_criterion="ei",
            n_points=1,
            seed=123,
            log_level=20,
            show_models=False,
            save_experiment=True,
            show_progress=True)
design_control = design_control_init(
            init_size=5,
            repeats=1)
surrogate_control = surrogate_control_init(
            model_fun_evals=10000,
            min_theta=-3,
            max_theta=3,
            theta_init_zero=True,
            n_p=1,
            optim_p=False,
            var_type=["num", "num"],
            seed=124)
optimizer_control = optimizer_control_init(
            max_iter=1000,
            seed=125)
S = Spot(fun=fun,
            fun_control=fun_control,
            design_control=design_control,
            surrogate_control=surrogate_control,
            optimizer_control=optimizer_control)
S.run()

In [ ]:
#| label: code-reload-optimization-experiment-604_2
S_exp = load_experiment(PREFIX=PREFIX)

In [ ]:
#| label: code-reload-optimization-experiment-604_3
S_res = load_result(PREFIX=PREFIX)

The progress of the original experiment is shown in @fig-plot-progress-604a_1 and the reloaded experiment in @fig-plot-progress-604b_2.

In [ ]:
#| label: fig-plot-progress-604a_1
#| fig-cap: Progress of the original experiment
S.plot_progress(log_y=True)

In [ ]:
#| label: fig-plot-progress-604b_2
#| fig-cap: Progress of the reloaded experiment
S_res.plot_progress(log_y=True)

The results from the original experiment are shown in @tbl-results-604a_1 and the reloaded experiment in @tbl-results-604b.

In [ ]:
#| label: tbl-results-604a_1
S.print_results()

In [ ]:
#| label: tbl-results-604b
S_res.print_results()

### Getting the Tuned Hyperparameters

The tuned hyperparameters can be obtained as a dictionary with the following code.
Since `spotpython` is used as an optimizer, the numerical levels of the hyperparameters are identical to the optimized values of the underlying optimization problem, here: the Branin function.

In [ ]:
#| label: code-get-tuned-optimization-604_11
get_tuned_hyperparameters(spot_tuner=S_res)

::: {.callout-note}
### Summary: Saving and Loading Optimization Experiments
* If `spotpython` is used as an optimizer (without an hyperparameter dictionary), experiments can be saved and reloaded with the `save_experiment` and `load_experiment` functions.
* The tuned hyperparameters can be obtained with the `get_tuned_hyperparameters` function.
:::

## spotpython as a Hyperparameter Tuner {#sec-spotpython-as-a-hyperparameter-tuner-604}

If `spotpython` is used as a hyperparameter tuner,
in addition to the `fun_control` dictionary a `core_model` dictionary has to be specified.
Furthermore, a data set has to be selected and added to the `fun_control` dictionary.
Here, we will use the `Diabetes` data set.


### The Diabetes Data Set

The hyperparameter tuning of a `PyTorch Lightning` network on the `Diabetes` data set is used as an example. The `Diabetes` data set is a PyTorch Dataset for regression, which originates from the `scikit-learn` package, see [https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_diabetes.html#sklearn.datasets.load_diabetes](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_diabetes.html#sklearn.datasets.load_diabetes).

Ten baseline variables, age, sex, body mass index, average blood pressure, and six blood serum measurements were obtained for each of n = 442 diabetes patients,  as well as the response of interest, a quantitative measure of disease progression one year after baseline.
The `Diabetes` data set is has the following properties:

* Samples total: 442
* Dimensionality: 10
* Features: real, $-.2 < x < .2$
* Targets: integer $25 - 346$

In [ ]:
#| label: code-diabetes-data-set-604_13
#| eval: true
data_set = Diabetes()

In [ ]:
#| label: code-hyperparameter-tuning-604_14
PREFIX="604"
fun_control = fun_control_init(
    save_experiment=True,
    PREFIX=PREFIX,
    fun_evals=inf,
    max_time=1,
    data_set = data_set,
    core_model_name="light.regression.NNLinearRegressor",
    hyperdict=LightHyperDict,
    _L_in=10,
    _L_out=1)

fun = HyperLight().fun

set_hyperparameter(fun_control, "optimizer", [ "Adadelta", "Adam", "Adamax"])
set_hyperparameter(fun_control, "l1", [3,4])
set_hyperparameter(fun_control, "epochs", [3,5])
set_hyperparameter(fun_control, "batch_size", [4,11])
set_hyperparameter(fun_control, "dropout_prob", [0.0, 0.025])
set_hyperparameter(fun_control, "patience", [2,3])

design_control = design_control_init(init_size=10)

print_exp_table(fun_control)

In contrast to the default setting, where `save_experiment` is set to `False`,
here the `fun_control` dictionary is initialized `save_experiment=True`.
Alternatively, an existing `fun_control` dictionary can be updated with `{"save_experiment": True}` as shown in the following code.

In [ ]:
#| label: 604_save_experiment_15
fun_control.update({"save_experiment": True})

If `save_experiment` is set to `True`, the results of the hyperparameter tuning experiment are stored in a pickle file with the name `PREFIX` after the tuning is finished in the current directory.

Alternatively, the spot object and the corresponding dictionaries can be saved with the `save_experiment` method, which is part of the `spot` object.
Therefore, the `spot` object has to be created as shown in the following code.

In [ ]:
S_diabetes = Spot(fun=fun,fun_control=fun_control, design_control=design_control)
S_diabetes.save_experiment(path="userExperiment", overwrite=False)

Here, we have added a `path` argument to specify the directory where the experiment is saved.
The resulting pickle file can be copied to another directory or computer and reloaded with the `load_experiment` function.
It can also be used for performing the tuning run.
Here, we will execute the tuning run on the local machine, which can be done with the following code.

In [ ]:
S_diabetes_res = S_diabetes.run()

After the tuning run is finished, a pickle file with the name `spot_604_experiment.pickle` is stored in the local directory.
This is a result of setting the `save_experiment` argument to `True` in the `fun_control` dictionary.
We can load the experiment with the following code. Here, we have specified the `PREFIX` as an argument to the `load_experiment` function.
Alternatively, the filename (`filename`) can be used as an argument.

In [ ]:
#| label: code-reload-hyper-experiment-37
S_diabetes_load_exp = load_experiment(PREFIX=PREFIX)
S_diabetes_load_res = load_result(PREFIX=PREFIX)

For comparison, the tuned hyperparameters of the original experiment are shown first: 

In [ ]:
#| label: code-get-tuned-hyperparameters-fun-ctrl604a
get_tuned_hyperparameters(S_diabetes, fun_control)

Second, the tuned hyperparameters of the reloaded experiment are shown:

In [ ]:
#| label: code-get-tuned-hyperparameters-fun-ctrl604b
get_tuned_hyperparameters(S_diabetes_load_res)

Note: The numerical levels of the hyperparameters are used as keys in the dictionary.
If the `fun_control` dictionary is used, the names of the hyperparameters are used as keys in the dictionary. 

In [ ]:
get_tuned_hyperparameters(S_diabetes_load_res, fun_control=S_diabetes_load_exp.fun_control)

Plot the progress of the original experiment are identical to the reloaded experiment.

In [ ]:
#| label: fig-plot-progress-604aa
S_diabetes.plot_progress()

In [ ]:
#| label: fig-plot-progress-604bb
S_diabetes_load_res.plot_progress()

::: {.callout-note}
### Summary: Saving and Loading Hyperparameter-Tuning Experiments
* If `spotpython` is used as an hyperparameter tuner (with an hyperparameter dictionary), experiments can be saved and reloaded with the `save_experiment` and `load_experiment` functions.
* The tuned hyperparameters can be obtained with the `get_tuned_hyperparameters` function.
:::


## Saving and Loading PyTorch Lightning Models {#sec-saving-and-loading-pytorch-lightning-models-604}

@sec-spotpython-saving-and-loading  and @sec-spotpython-as-a-hyperparameter-tuner-604 explained how to save and load optimization and hyperparameter tuning experiments and how to get the tuned hyperparameters as a dictionary.
This section shows how to save and load `PyTorch Lightning` models.


### Get the Tuned Architecture {#sec-get-spot-results-604}

In contrast to the function `get_tuned_hyperparameters`, the function `get_tuned_architecture` returns the tuned architecture of the model as a dictionary. Here, the transformations are already applied to the numerical levels of the hyperparameters and the encoding (and types) are the original types of the hyperparameters used by the model.
Important: The `config` dictionary from `get_tuned_architecture` can be passed to the model without any modifications.

In [ ]:
config = get_tuned_architecture(S_diabetes)
pprint.pprint(config)

After getting the tuned architecture, the model can be created and tested with the following code.

In [ ]:
test_model(config, fun_control)

### Load a Model from Checkpoint

The method `load_light_from_checkpoint` loads a model from a checkpoint file.
Important: The model has to be trained before the checkpoint is loaded. As shown here, loading a model with trained weights is possible, but requires two steps:

1. The model weights have  to be learned using `test_model`. The `test_model` method writes a checkpoint file.
2. The model has to be loaded from the checkpoint file.

#### Details About the `load_light_from_checkpoint` Method

* The `test_model` method saves the last checkpoint to a file using the following code:
```python
ModelCheckpoint(
    dirpath=os.path.join(fun_control["CHECKPOINT_PATH"], config_id), save_last=True
), 
```

The filename of the last checkpoint has a specific structure:

* A `config_id` is generated from the `config` dictionary. It does not use a timestamp. This differs from the config id generated in cvmodel.py and trainmodel.py, which provide time information for the TensorBoard logging.
* Furthermore, the postfix `_TEST` is added to the `config_id` to indicate that the model is tested.
* For example: `runs/saved_models/16_16_64_LeakyReLU_Adadelta_0.0014_8.5895_8_False_kaiming_uniform_TEST/last.ckpt`

In [ ]:
model_loaded = load_light_from_checkpoint(config, fun_control)

In [ ]:
vars(model_loaded)

In [ ]:
torch.save(model_loaded, "model.pt")

Note the following warning:
In PyTorch 2.6, the default value of the `weights_only` argument in `torch.load`  was changed from `False` to `True`. Re-running `torch.load` with `weights_only` set to `False` will likely succeed, but it can result in arbitrary code execution. Do it only if you got the file from a trusted source.

In [ ]:
mymodel = torch.load("model.pt", weights_only=False)

Show all attributes of the model:

In [ ]:
vars(mymodel)

## Converting a Lightning Model to a Plain Torch Model {#sec-converting-a-lightning-model-to-a-plain-torch-model-604}

### The Function `get_removed_attributes_and_base_net`

`spotpython` provides a function to covert a `PyTorch Lightning` model to a plain `PyTorch` model. The function `get_removed_attributes_and_base_net` returns a tuple with the removed attributes and the base net. The base net is a plain `PyTorch` model. The removed attributes are the attributes of the `PyTorch Lightning` model that are not part of the base net.

This conversion can be reverted.

In [ ]:
removed_attributes, torch_net = get_removed_attributes_and_base_net(net=mymodel)

In [ ]:
print(removed_attributes)

In [ ]:
print(torch_net)

###  An Example how to use the Plain Torch Net

In [ ]:
# Load the Diabetes dataset from sklearn
diabetes = load_diabetes()
X = diabetes.data
y = diabetes.target

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Convert the data to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

# Create a PyTorch dataset
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

# Create a PyTorch dataloader
batch_size = 32
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size)

torch_net.to(getDevice("cpu"))

# train the net
criterion = nn.MSELoss()
optimizer = optim.Adam(torch_net.parameters(), lr=0.01)
n_epochs = 100
losses = []
for epoch in range(n_epochs):
    for inputs, targets in train_dataloader:
        targets = targets.view(-1, 1)
        optimizer.zero_grad()
        outputs = torch_net(inputs)
        loss = criterion(outputs, targets)
        losses.append(loss.item())
        loss.backward()
        optimizer.step()
# visualize the network training
plt.plot(losses)
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.show()